# Reference Tiles

In [ ]:
from pathlib import Path
import warnings
import geopandas as gpd
import matplotlib.pyplot as plt

from vegmapper import pathurl
from vegmapper.core.prep_tiles import prep_tiles

## User Inputs
In the cell below, make sure to define the following paths
- Define site name.
- Locate or create a project directory.
- Define the path of the site boundary layer.
  - Currently expected to be inside the project directory 

In [ ]:
# Site name
sitename = 'ucayali'

# Project directory (local path or cloud bucket URL)
proj_dir = './ucayali'

# AOI file
aoifile = f'{proj_dir}/ucayali_boundary.geojson'

In [ ]:
# create project directory
proj_dir_path = Path(proj_dir)
# Create the directory if it doesn't exist
if not proj_dir_path.exists():
    proj_dir_path.mkdir(parents=True, exist_ok=True)
    print(f"Directory created: {proj_dir_path}")
else:
    print(f"Directory already exists: {proj_dir_path}")

# verify that the aoifile exits.
warnings.simplefilter("always", UserWarning)
aoifile_path = Path(aoifile)
if aoifile_path.is_file():
    print(f"Boundary file found: {aoifile_path}")
else:
    # Raise a warning if the file does not exist
    warnings.warn(
        f"Boundary file {aoifile_path} was not found or does not exist, "
        "please make sure the file path is correct before continuing",
        UserWarning
    )

## `prep_tiles`

To create a data stack, a reference tiling system is required to ensure all data layers are aligned to the same tiles on the same projection (UTM). A GeoJSON file containing the polygons of the tiles is generated by the function `vegmapper.prep_tiles`.

### Usage

```
vegmapper.prep_tiles(aoi_name, aoi_boundary, tile_size)
```

|Arguments|Description|Required|Default|
|----|----|----|----|
|`aoi_name`|Used to name the output geoJSON. The generated tiles can be found at `aoi_name`_tiles.geojson.|Yes|None|
|`aoi_boundary`|shp/geoJSON file that contains the boundaries of our area of interest. geoJSON files for many subnational administrative boundaries can be found [here](https://data.humdata.org/dataset?ext_subnational=1&res_format=GeoJSON&vocab_Topics=administrative%20divisions&q=&sort=if(gt(last_modified%2Creview_date)%2Clast_modified%2Creview_date)%20desc&ext_page_size=25).|Yes|None|
|`tile_size`|The desired tile size, in meters.|Yes|None|

In [ ]:
# Select desired tile size in meters
tile_size = 150000

Generate the UTM tiles by running the cell below:

In [ ]:
aoi_tiles = prep_tiles(sitename, aoifile, tile_size, centered=True)

In [ ]:
# Read in tiles and boundary
gdf_tiles = gpd.read_file(aoi_tiles)
gdf_aoi = gpd.read_file(aoifile).dissolve()
gdf_aoi_utm = gdf_aoi.to_crs(gdf_tiles.crs)

# Plot tiles
ax = gdf_tiles.boundary.plot(figsize=(10, 10), color='black')
# Plot boundary
gdf_aoi_utm.plot(ax=ax, color='blue')
# Plot tiles not intersecting AOI in gray
if (gdf_tiles['mask'] == 0).any():
    ax = gdf_tiles[gdf_tiles['mask'] == 0].plot(ax=ax, color='gray')

The tiles not intersecting AOI are shown in gray color. The final data stacks will not be generated for these tiles.